In [0]:
install.packages("mlflow")
install.packages("carrier")

* installing *source* package ‘mlflow’ ...
** package ‘mlflow’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (mlflow)
* installing *source* package ‘carrier’ ...
** package ‘carrier’ successfully unpacked and MD5 sums checked
** using staged installation
** R
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (carrier)
Installing package into

In [0]:
# Load required libraries
library(sparklyr)
library(dplyr)
library(mlflow)
library(rpart)

In [0]:
# 1. Read the iris_data table from Unity Catalog into a dataframe

# Read the iris data from the specified table
iris_tbl <- tbl(spark, "pedroz_e2edata_dev.default.iris_features")

# Collect data into R for modeling
iris_data <- collect(iris_tbl)

In [0]:
# 2. Load the model from Unity Catalog using mlflow_load_model
model <- mlflow_load_model("models:/pedroz_e2edata_dev.default.iris_r_class_model_uc@champion")

In [0]:
model

<crate> 51.46 kB
* function: 1.18 kB
* `model`: 49.89 kB
function (newdata) 
{
    stats::predict(model, newdata = newdata, type = "class")
}

In [0]:
# model is the crate you loaded
preds <- model(iris_data)


In [0]:
# Add predictions to the iris_data dataframe
iris_data$prediction <- preds

# Copy iris_data with predictions back to Spark
iris_pred_tbl <- copy_to(spark, iris_data, "iris_data_with_preds", overwrite = TRUE)

In [0]:
display(iris_pred_tbl)

sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,id,prediction
5.1,3.5,1.4,0.2,1.0,0
4.9,3.0,1.4,0.2,2.0,0
4.7,3.2,1.3,0.2,3.0,0
4.6,3.1,1.5,0.2,4.0,0
5.0,3.6,1.4,0.2,5.0,0
5.4,3.9,1.7,0.4,6.0,0
4.6,3.4,1.4,0.3,7.0,0
5.0,3.4,1.5,0.2,8.0,0
4.4,2.9,1.4,0.2,9.0,0
4.9,3.1,1.5,0.1,10.0,0


In [0]:
# Save the Spark DataFrame to a Unity Catalog table using sparklyr
spark_write_table(
  iris_pred_tbl,
  name = "pedroz_e2edata_dev.default.iris_data_with_preds",
  mode = "overwrite"
)